In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.cross_validation import train_test_split

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:

example=pd.read_csv('example.csv')


train_df, test_df = train_test_split(example, train_size = 0.8)
X_train = train_df.drop([ 'Grant.Status'], axis=1)
y_train = train_df['Grant.Status']
X_test = test_df.drop([ 'Grant.Status'], axis=1)
y_test = test_df['Grant.Status']


dtrain = xgb.DMatrix(X_train.values,y_train.values)
dtest = xgb.DMatrix(X_test.values, y_test.values)
param = {'max_depth':7, 'silent': 1, 'objective':'binary:logistic'}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 30

xgb_train = xgb.train(param, dtrain, num_round)

y_pred = xgb_train.predict(dtrain, pred_leaf=True)
num_leaf= 610
X_train_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
for i in range(0,len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
    X_train_leaves[i][temp] += 1
    
y_pred= xgb_train.predict(dtest, pred_leaf=True)    
X_test_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
for i in range(0,len(y_pred)):
    temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
    X_test_leaves[i][temp] += 1
    

In [4]:
lr = LogisticRegression()
lr.fit(X_train_leaves, y_train)
y_pred_gbdtlr1 = lr.predict(X_test_leaves)
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

gbdt+lr auc 1: 0.85259


In [72]:
len(y_pred)

5794

In [9]:
np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[0])

array([  154,   764,  1385,  2022,  2594,  3217,  3830,  4436,  5049,
        5649,  6216,  6836,  7450,  8000,  8619,  9283,  9830, 10495,
       11104, 11675, 12270, 12868, 13516, 14115, 14778, 15336, 15893,
       16579, 17196, 17803])